In [10]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [11]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo
WHERE repo_name = 'concourse' OR repo_name = 'postfacto' or repo_name = 'clarity' or repo_name = 'gpdb' or
      repo_name = 'kpack' or repo_name = 'rabbitmq-server' or repo_name = 'sonobuoy' or repo_name = 'velero' 
      or repo_name = 'octant' or repo_name = 'gpdb' or repo_name = 'tern';
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)

    repo_id        repo_name                              repo_path
0     26983        postfacto                    github.com/pivotal/
1     26235        concourse               github.com/pcfdev-forks/
2     25432  rabbitmq-server                   github.com/rabbitmq/
3     25857             gpdb               github.com/greenplum-db/
4     28037           octant               github.com/vmware-tanzu/
5     28031           velero               github.com/vmware-tanzu/
6     28030         sonobuoy               github.com/vmware-tanzu/
7     26600             gpdb  github.com/Pivotal-Field-Engineering/
8     27169             gpdb                github.com/pivotal-gss/
9     28051        concourse                  github.com/concourse/
10    27043            kpack                    github.com/pivotal/
11    27913          clarity                     github.com/vmware/
12    27940             tern                 github.com/tern-tools/


In [12]:
#from datetime import date
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
print(today)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"
print(end_date)

print

start = last_month - datetime.timedelta(days=365)
year_ago = "'" + str(start) + "'"
print(year_ago)

'2020-04-22'
'2020-03-31'
'2019-04-01'


In [18]:
value = 25432
affil_query = f"""
    select distinct(count(cmt_commit_hash)), cmt_author_affiliation
    from commits
    where 
        repo_id = {value}
        AND cmt_author_timestamp >= {year_ago}
        AND cmt_author_timestamp <= {end_date}
    group by cmt_author_affiliation;
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

   count cmt_author_affiliation
0      1                Red Hat
1    182                 VMware
2    344                   NULL
3    957              (Unknown)


In [14]:
affil_query = f"""
    select distinct(count(cmt_commit_hash)), cmt_author_affiliation
    from commits
    where 
        cmt_author_timestamp >= {year_ago}
        AND cmt_author_timestamp <= {end_date}
    group by cmt_author_affiliation;
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

     count                  cmt_author_affiliation
0        1                                   Intel
1        1                                    Nike
2        1               State Farm Insurance Cos.
3        1                      State Street Corp.
4        1                          Voya Financial
5        2                                Allstate
6        2                                    eBay
7        2  Fidelity National Information Services
8        2                              Ford Motor
9        2                                McKesson
10       2                          salesforce.com
11       3                                   Merck
12       4                                    AT&T
13       4                                Free BSD
14       5                            Ingram Micro
15       5                                 Samsung
16       6                                   Apple
17      11                  University of Missouri
18      22                     

In [17]:
affil_query = f"""
    select distinct(count(cmt_commit_hash)), cmt_author_affiliation
    from commits
    where 
        cmt_author_timestamp >= {year_ago}
        AND cmt_author_timestamp <= {end_date}
        AND cmt_author_email LIKE '%%bitnami%%'
    group by cmt_author_affiliation;
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

    count cmt_author_affiliation
0    2582              (Unknown)
1  268102                   NULL


In [12]:
affil_all_query = f"""
    select cmt_author_name, cmt_author_affiliation, cmt_author_email
    from commits
    group by cmt_author_name, cmt_author_affiliation, cmt_author_email;
    """
affil_all = pd.read_sql_query(affil_all_query, con=engine)
print(affil_all)

              cmt_author_name cmt_author_affiliation  \
0            InvalidInterrupt              (Unknown)   
1          Bartosz Grzybowski              (Unknown)   
2                 Carson Wang                  Intel   
3          Matt Braymer-Hayes              (Unknown)   
4                 Milan Plžík              (Unknown)   
...                       ...                    ...   
35932  Mikhail Vyshegorodtsev                 VMware   
35933            Josh Trotter              (Unknown)   
35934         Martin Lundberg              (Unknown)   
35935       Evgeny Egorochkin              (Unknown)   
35936                   Ram V              (Unknown)   

                                  cmt_author_email  
0        InvalidInterrupt@users.noreply.github.com  
1                                melkorm@gmail.com  
2                            carson.wang@intel.com  
3                           matt.hayes91@gmail.com  
4                  emempi@users.noreply.github.com  
...      

In [20]:
affil_by_person_query = f"""
    select cmt_author_name, cmt_author_affiliation, cmt_author_email
    from commits
    where cmt_author_email like '%%asuraci%%'
    group by cmt_author_name, cmt_author_affiliation, cmt_author_email;
    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)

                                  cmt_author_name cmt_author_affiliation  \
0                    Alex Suraci and David Varvel                 VMware   
1                Alex Suraci and Maria Shaldibina                 VMware   
2                   Alex Suraci and Onsi Fakhouri                 VMware   
3                      Alex Suraci and Ruth Byers                 VMware   
4  Alex Suraci, Michael Maximilien and Ruth Byers                 VMware   

                            cmt_author_email  
0            asuraci+dvarvel@pivotallabs.com  
1            asuraci+mariash@pivotallabs.com  
2               asuraci+onsi@pivotallabs.com  
3             asuraci+rbyers@pivotallabs.com  
4  asuraci+maximilien+rbyers@pivotallabs.com  


In [7]:
affil_by_person_query = f"""
                    SELECT
                        DISTINCT(cmt_commit_hash),
                        CASE WHEN contributors.cntrb_canonical IS NOT NULL THEN contributors.cntrb_canonical ELSE cmt_author_email END AS cntrb_canonical,
                        CASE WHEN canonical_full_names.cntrb_full_name IS NOT NULL THEN canonical_full_names.cntrb_full_name ELSE cmt_author_name END AS canonical_full_name,
                        cmt_author_name, cmt_author_email, cmt_author_affiliation, repo_id, cmt_author_timestamp 
                    FROM commits 
                        LEFT OUTER JOIN contributors ON cntrb_email = cmt_author_email
                        LEFT OUTER JOIN (
                            SELECT cntrb_canonical, cntrb_full_name 
                            FROM contributors
                            WHERE cntrb_canonical = cntrb_email
                        ) canonical_full_names
                        ON canonical_full_names.cntrb_canonical = contributors.cntrb_canonical
                    WHERE
                        repo_id = {value}
                        AND cmt_author_name NOT LIKE 'snyk%%'
                        AND cmt_author_name NOT LIKE '%%bot'
                        AND cmt_author_name != 'Spring Operator'
                        AND cmt_author_name != 'Spring Buildmaster'
                         AND cmt_author_timestamp >= {year_ago}
                         AND cmt_author_timestamp <= {end_date}
                    ORDER BY
                        cmt_author_affiliation;
                    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
#print(affil_by_person)
affil_group = affil_by_person.groupby(['cmt_author_affiliation', 'canonical_full_name'], as_index=False).sum()[['cmt_author_affiliation', 'canonical_full_name']]
print(affil_group)


   cmt_author_affiliation canonical_full_name
0               (Unknown)       Administrator
1               (Unknown)   Alessandro Degano
2               (Unknown)         Alex Suraci
3               (Unknown)    Alexander Gurney
4               (Unknown)         Aman Wangde
..                    ...                 ...
93                 VMware            Rui Yang
94                 VMware        Sameer Vohra
95                 VMware        Taylor Silva
96                 VMware      Topher Bullock
97                 VMware         Vlad Stoian

[98 rows x 2 columns]


In [8]:
authorDF = pd.DataFrame()
authorDF = affil_by_person.cmt_author_affiliation.value_counts()
authorDF = authorDF.reset_index()
authorDF.columns = ['name', 'commits']
print(authorDF.head(10))

        name  commits
0     VMware     1501
1  (Unknown)      858
2       NULL       55
3        ADP       10


# Old Stuff

In [2]:
affil_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'contributors';
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

               column_name
0                 cntrb_id
1              cntrb_login
2              cntrb_email
3            cntrb_company
4         cntrb_created_at
5               cntrb_type
6               cntrb_fake
7            cntrb_deleted
8               cntrb_long
9                cntrb_lat
10      cntrb_country_code
11             cntrb_state
12              cntrb_city
13          cntrb_location
14         cntrb_canonical
15              gh_user_id
16                gh_login
17                  gh_url
18             gh_html_url
19              gh_node_id
20           gh_avatar_url
21          gh_gravatar_id
22        gh_followers_url
23        gh_following_url
24            gh_gists_url
25          gh_starred_url
26    gh_subscriptions_url
27    gh_organizations_url
28            gh_repos_url
29           gh_events_url
30  gh_received_events_url
31                 gh_type
32           gh_site_admin
33             tool_source
34            tool_version
35             data_source
3

In [3]:
affilx_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'contributors_aliases';
    """
affilx = pd.read_sql_query(affilx_query, con=engine)
print(affilx)

            column_name
0              cntrb_id
1            cntrb_a_id
2       canonical_email
3           alias_email
4          cntrb_active
5   cntrb_last_modified
6           tool_source
7          tool_version
8           data_source
9  data_collection_date


In [4]:
affilx2_query = f"""
SELECT
   canonical_email, alias_email
FROM
   contributors_aliases
;
    """
affilx2 = pd.read_sql_query(affilx2_query, con=engine)
print(affilx2)

                        canonical_email                  alias_email
0                mikeb@squaremobius.net           mikeb@rabbitmq.com
1                mikeb@squaremobius.net             mikeb@lshift.net
2     jean-sebastien.pedron@dumbbell.fr  jean-sebastien@rabbitmq.com
3              michael@clojurewerkz.org         michael@rabbitmq.com
4                           sam@soff.es                  sam@soff.es
...                                 ...                          ...
2628      paul.johnston@portswigger.net               pcj@pubref.org
2629              kevin.clark@gmail.com        kevin.clark@gmail.com
2630              nfirvine@nfirvine.com        nfirvine@nfirvine.com
2631              jasonyoung@google.com        jasonyoung@google.com
2632              loic.porte@bibabox.fr        loic.porte@bibabox.fr

[2633 rows x 2 columns]


In [33]:
affil2_query = f"""
SELECT
   ca_domain, ca_affiliation
FROM
   contributor_affiliations
;
    """
affil2 = pd.read_sql_query(affil2_query, con=engine)
print(affil2)

               ca_domain          ca_affiliation
0            samsung.com                 Samsung
1    linuxfoundation.org        Linux Foundation
2                ibm.com                     IBM
3            walmart.com                 Walmart
4         exxonmobil.com             Exxon Mobil
..                   ...                     ...
515          twitter.com                 Twitter
516            adobe.com                   Adobe
517              acm.org                     ACM
518     outdoors@acm.org  University of Missouri
519          freebsd.org                Free BSD

[520 rows x 2 columns]


In [70]:
affil_by_person_query = f"""
    select distinct(count(cmt_commit_hash))
    from commits
    where cmt_author_email = 'soumyadeep2007@gmail.com';
    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)

   count
0      0


In [64]:
affil_by_person_query = f"""
    select cmt_author_name, cmt_author_affiliation, cmt_author_email
    from commits
    where cmt_author_name like '%%rraca%%'
    group by cmt_author_name, cmt_author_affiliation, cmt_author_email;
    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)

  cmt_author_name cmt_author_affiliation cmt_author_email
0     Luis Urraca              (Unknown)   me@lurraca.com


In [28]:
affil3_query = f"""
SELECT
   cntrb_company, cntrb_email, cntrb_canonical, cntrb_full_name, cntrb_login
FROM
   contributors
WHERE
   cntrb_company LIKE '%%pivotal%%'
   AND (cntrb_email not LIKE '%%pivotal%%' and cntrb_canonical not LIKE '%%pivotal%%')
GROUP BY
   cntrb_company, cntrb_email, cntrb_canonical, cntrb_full_name, cntrb_login
;
    """
affil3 = pd.read_sql_query(affil3_query, con=engine)
affil3[0:20]

,cntrb_company,cntrb_email,cntrb_canonical,cntrb_full_name,cntrb_login
0,@pivotal,alex@localhost.localdomain,suraci.alex@gmail.com,Alex Suraci,vito
1,@pivotal,btong@berkeley.edu,btong@berkeley.edu,None,bbtong
2,@pivotal,delagrammatikas@gmail.com,delagrammatikas@gmail.com,Marc Delagrammatikas,marcdel
3,@pivotal,hello@motevets.com,hello@motevets.com,None,motevets
4,@pivotal,julian.zucker@gmail.com,julian.zucker@gmail.com,Julian Zucker,julian-zucker
5,@pivotal,sahil.muthoo@gmail.com,sahil.muthoo@gmail.com,Sahil Muthoo,sahilm
6,@pivotal,suraci.alex@gmail.com,suraci.alex@gmail.com,Alex Suraci,vito
7,@pivotal,tanayk2610@gmail.com,tanayk2610@gmail.com,Tanay Kothari,tanayk2610
8,@pivotal,aussielunix@gmail.com,aussielunix@gmail.com,Mick Pollard,aussielunix
9,@pivotal,contact@rizwanreza.com,contact@rizwanreza.com,None,rizwanreza


In [40]:
affil4_query = f"""
SELECT
   cntrb_email, cntrb_type
FROM
   contributors
WHERE
   cntrb_company LIKE '%%pivotal%%' 
GROUP BY
   cntrb_email, cntrb_type
;
    """
affil4 = pd.read_sql_query(affil4_query, con=engine)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(affil4)

                                       cntrb_email cntrb_type
0                           abhayashenoy@gmail.com       None
1                                abrown@pivotal.io       None
2                             acbdesigns@gmail.com       None
3                              aedstrom@pivotal.io       None
4                           ajaganathan@pivotal.io       None
5           ajaganathan@pivotals-MacBook-Pro.local       None
6                         alexey.nesterov@live.com       None
7                       alex@localhost.localdomain       None
8                           alfusinigo.j@gmail.com       None
9                          andrewedstrom@gmail.com       None
10                               andymoe@gmail.com       None
11                 aniruth.parthasarathy@gmail.com       None
12                              aslynko@pivotal.io       None
13                             astandke@pivotal.io       None
14                              asuraci@pivotal.io       None
15      

In [9]:
affil5_query = f"""
SELECT
   cntrb_email
FROM
   contributors
WHERE
   cntrb_email LIKE '%%hept%%' 
GROUP BY
   cntrb_email
;
    """
affil5 = pd.read_sql_query(affil5_query, con=engine)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(affil5)

                  cntrb_email
0             adam@hepton.org
1          asenchi@heptio.com
2            chuck@heptio.com
3          dcooley@heptio.com
4            derek@heptio.com
5             eric@heptio.com
6   jesse.hamilton@heptio.com
7              joe@heptio.com
8            jorge@heptio.com
9      mlandaverde@heptio.com
10        nicholas@heptio.com
11           nolan@heptio.com
12          steves@heptio.com
